# LOADING CSV

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/06-loading-structured-data.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/06-loading-structured-data.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/06-loading-structured-data.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table><br><br>

### Connect to EvaDB

In [1]:
%pip install --quiet "evadb[vision,notebook]"
import evadb
cursor = evadb.connect().cursor()

Note: you may need to restart the kernel to use updated packages.


### Create a Meta-data Table

In [2]:
create_table_query = f"""
    CREATE TABLE IF NOT EXISTS bddtestmeta(
        id INTEGER UNIQUE,
        frame_id INTEGER,
        video_id INTEGER,
        dataset_name TEXT(30),
        label TEXT(30),
        bbox NDARRAY FLOAT32(4),
        object_id INTEGER
    );
"""
cursor.query(create_table_query).df()

09-05-2023 14:44:57 WARNING[executor_utils:executor_utils.py:handle_if_not_exists:0084] Table: bddtestmeta already exists


""


### Download Berkeley Deepdrive Videos

In [3]:
# sample dataset of 2 videos
!wget -nc "https://www.dropbox.com/s/kg1q69ijbhjfecw/bddtest.zip"

# unzip
!unzip -n bddtest.zip

File ‘bddtest.zip’ already there; not retrieving.



Archive:  bddtest.zip


### Load 2 CSV files

In [4]:
cursor.query("LOAD CSV './bddtest/info/bddtest_00a2e3ca5c856cde.csv' INTO bddtestmeta").df()
cursor.query("LOAD CSV './bddtest/info/bddtest_00a395fed60c0b47.csv' INTO bddtestmeta").df()

,CSV,Number of loaded frames
0,bddtest/info/bddtest_00a395fed60c0b47.csv,4355


## Load the videos

In [5]:
cursor.query('DROP TABLE IF EXISTS bddtest_1;').df()
cursor.query('DROP TABLE IF EXISTS bddtest_2;').df()

cursor.query("LOAD VIDEO './bddtest/videos/bddtest_00a2e3ca5c856cde.mp4' INTO bddtest_1").df()
cursor.query("LOAD VIDEO './bddtest/videos/bddtest_00a395fed60c0b47.mp4' INTO bddtest_2").df()

,0
0,Number of loaded VIDEO: 1


## Run a Query over the Video dataset


In [6]:
query = cursor.query("""
    SELECT id, Yolo(data)
    FROM bddtest_1
    WHERE id < 3
""")
query.df()

/Users/jiashenc/Workspace/eva/evadb_venv/lib/python3.11/site-packages/evadb/catalog/services/column_catalog_service.py:59: SAWarning: Object of type <UdfCacheCatalog> not in session, add operation along 'UdfCatalog._dep_caches' won't proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation.)
  entry = self.session.execute(


/Users/jiashenc/Workspace/eva/evadb_venv/lib/python3.11/site-packages/evadb/models/storage/batch.py:271: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  new_frames = pd.concat(frames, axis=1, copy=False, ignore_index=False).fillna(
/Users/jiashenc/Workspace/eva/evadb_venv/lib/python3.11/site-packages/evadb/models/storage/batch.py:271: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  new_frames = pd.concat(frames, axis=1, copy=False, ignore_index=False).fillna(


/Users/jiashenc/Workspace/eva/evadb_venv/lib/python3.11/site-packages/evadb/models/storage/batch.py:271: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  new_frames = pd.concat(frames, axis=1, copy=False, ignore_index=False).fillna(


,bddtest_1.id,yolo.labels,yolo.bboxes,yolo.scores
0,0,"[car, stop sign, car, car, car, car, car, car,...","[[490.39593505859375, 332.89349365234375, 716....","[0.93, 0.91, 0.88, 0.82, 0.77, 0.73, 0.72, 0.5..."
1,1,"[car, car, stop sign, car, car, car, car, car,...","[[486.3235168457031, 331.76837158203125, 715.9...","[0.92, 0.87, 0.85, 0.77, 0.74, 0.7, 0.62, 0.56..."
2,2,"[car, stop sign, car, car, car, car, car, car,...","[[481.6331481933594, 331.23248291015625, 715.7...","[0.93, 0.9, 0.87, 0.78, 0.77, 0.74, 0.57, 0.54..."


## Run a query over the Meta-data Table

In [7]:
query = cursor.query("""
    SELECT *
    FROM bddtestmeta
    WHERE bddtestmeta.video_id = 4
""")
query.df()

/Users/jiashenc/Workspace/eva/evadb_venv/lib/python3.11/site-packages/evadb/models/storage/batch.py:271: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  new_frames = pd.concat(frames, axis=1, copy=False, ignore_index=False).fillna(


/Users/jiashenc/Workspace/eva/evadb_venv/lib/python3.11/site-packages/evadb/models/storage/batch.py:271: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  new_frames = pd.concat(frames, axis=1, copy=False, ignore_index=False).fillna(


,bddtestmeta._row_id,bddtestmeta.id,bddtestmeta.frame_id,bddtestmeta.video_id,bddtestmeta.dataset_name,bddtestmeta.label,bddtestmeta.bbox,bddtestmeta.object_id
0,1,8185,0,4,bddtest,car,"[491.7783, 320.9238, 717.0901, 512.1478]",512
1,2,8186,0,4,bddtest,car,"[928.2679, 298.47574, 1279.5381, 571.17786]",513
2,3,8187,0,4,bddtest,car,"[816.0277, 347.52887, 1029.6998, 474.7344]",514
3,4,8188,0,4,bddtest,car,"[811.87067, 325.91223, 948.2217, 408.2217]",515
4,5,8189,0,4,bddtest,car,"[673.0254, 325.08084, 751.17786, 367.48267]",516
...,...,...,...,...,...,...,...,...
13599,26665,11581,1197,4,bddtest,car,"[554.96533, 335.88916, 588.2217, 378.291]",643
13600,26666,11582,1197,4,bddtest,car,"[693.8106, 350.0231, 734.5497, 395.75058]",644
13601,26667,11583,1197,4,bddtest,car,"[683.83374, 345.03464, 733.71826, 389.93073]",642
13602,26668,11584,1197,4,bddtest,car,"[676.351, 346.69745, 699.6305, 384.11084]",641
